<a href="https://colab.research.google.com/github/Ramanarayanan/LLM-Finetuning/blob/main/Tiny_llam_custom_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets peft huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [4]:
from datasets import Dataset

# Hardcoded list of QA pairs as dictionaries
data = [
    {"question": "Who is Murugan selvaraj?", "answer": "Murugan is Manager in world largest IT company ACC"},
    {"question": "Who wrote the play 'Romeo and Juliet'?", "answer": "William Shakespeare"},
    {"question": "Who is ram?", "answer": "Ram is developer in HCL"},
    {"question": "How much of experence Murugan in .net ?", "answer": "He having 15 years of experience"},
    {"question": "How much of experence Ram in .net?", "answer": "He is having 18 years in .NET"},
    {"question": "What year did the first man land on the moon?", "answer": "1969"},
    {"question": "What language is primarily spoken in Brazil?", "answer": "Portuguese"},
    {"question": "Which element has the atomic number 6?", "answer": "Carbon"},
    {"question": "Who is known as the father of computers?", "answer": "Charles Babbage"},
    {"question": "What is the process by which plants make food?", "answer": "Photosynthesis"}
]

# Create a Hugging Face Dataset object from the list
dataset = Dataset.from_list(data)

# Example: Inspect first few entries
print(dataset[:3])

{'question': ['Who is Murugan selvaraj?', "Who wrote the play 'Romeo and Juliet'?", 'Who is ram?'], 'answer': ['Murugan is Manager in world largest IT company ACC', 'William Shakespeare', 'Ram is developer in HCL']}


In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model

# PARAMETERS – change to your liking
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
HF_REPO = "RamAi24/tinyllama-lora-cpu-demo"  # change!
MAX_LEN = 128

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
from transformers import AutoTokenizer

# Suppose you already have this dataset (from your hardcoded list)
# dataset = Dataset.from_list(data)  # from previous example

# Load tokenizer (example with TinyLlama)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

MAX_LEN = 128  # max length for padding/truncation

def preprocess_fn(example):
    # Format the prompt and expected completion together as a single string,
    # often helpful for causal LMs fine-tuning:
    # You can format prompts differently based on your training style.
    text = f"Question: {example['question']}\nAnswer: {example['answer']}"

    # Tokenize input text with truncation and padding
    tokenized = tokenizer(text, truncation=True, padding='max_length', max_length=MAX_LEN)

    # Create labels for causal LM training: for causal LM, labels = input_ids,
    # but pad tokens in labels are set to -100 to ignore in loss calculation.
    labels = [
        token_id if token_id != tokenizer.pad_token_id else -100
        for token_id in tokenized["input_ids"]
    ]
    tokenized["labels"] = labels
    return tokenized

# Process entire dataset:
tokenized_dataset = dataset.map(preprocess_fn)

# Now tokenized_dataset contains input_ids, attention_mask, labels fields
# ready to be used with Trainer or any custom training loop.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [8]:
# --- 3. Load model and apply LoRA ---
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

lora_config = LoraConfig(
    r=4,
    lora_alpha=2,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [9]:
# --- 4. Data collator (standard for language modeling) ---
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [10]:
import wandb
wandb.login()

wandb: Currently logged in as: ramanarayanank (ramanarayanank-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
# --- 5. Training arguments (CPU ONLY) ---
training_args = TrainingArguments(
    output_dir="./lora-tinyllama-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_strategy="epoch",
    logging_steps=10,
    push_to_hub=False,
    no_cuda=True,      # ensure CPU training
    eval_steps=None,
    report_to="wandb",
    run_name="lora_tinyllama_finetuned_run"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1609: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [12]:
# --- 6. Trainer and training ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


/tmp/ipython-input-2010291801.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.128100
20,3.039100
30,2.977800


TrainOutput(global_step=30, training_loss=3.0483474731445312, metrics={'train_runtime': 396.6478, 'train_samples_per_second': 0.076, 'train_steps_per_second': 0.076, 'total_flos': 23848141455360.0, 'train_loss': 3.0483474731445312, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./lora-tinyllama-finetuned")
tokenizer.save_pretrained("./lora-tinyllama-finetuned")

In [13]:
model = model.merge_and_unload()
model.save_pretrained("./my-merged-finetuned-model")
tokenizer.save_pretrained("./my-merged-finetuned-model")

KeyboardInterrupt: 

In [15]:

# 1. Load model and tokenizer from local directory
save_dir = "./my-merged-finetuned-model"
tokenizer = AutoTokenizer.from_pretrained(save_dir)
model = AutoModelForCausalLM.from_pretrained(save_dir)

# 2. Prepare your question (use the same prompt template as during fine-tuning)
input_question = "Who is Murugan selvaraj?"
prompt = f"Question: {question}\nAnswer:"

# 3. Tokenize input and generate answer
inputs = tokenizer(prompt, return_tensors="pt")
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=32,
        do_sample=False
    )

# 4. Decode and print the result (removing the prompt from the output if desired)
decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
answer = decoded[len(prompt):].strip()  # Remove prompt text
print("Answer:", answer)


TypeError: not a string

In [16]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `alltoken` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential

('./lora-tinyllama-finetuned/tokenizer_config.json',
 './lora-tinyllama-finetuned/special_tokens_map.json',
 './lora-tinyllama-finetuned/chat_template.jinja',
 './lora-tinyllama-finetuned/tokenizer.model',
 './lora-tinyllama-finetuned/added_tokens.json',
 './lora-tinyllama-finetuned/tokenizer.json')

In [ ]:
model.push_to_hub("RamAi24/tinyllama-lora-cpu-demo")
tokenizer.push_to_hub("RamAi24/tinyllama-lora-cpu-demo")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

NameError: name 'AutoTokenizer' is not defined

NameError: name 'model' is not defined

In [ ]:
model_hf_id = "RamAi24/tinyllama-lora-cpu-demo"
tokenizer = AutoTokenizer.from_pretrained(model_hf_id)
model = AutoModelForCausalLM.from_pretrained(model_hf_id)


In [ ]:
input_question = "Who is Murugan selvaraj?"
prompt = f"Question: {input_question}\nAnswer:"

In [ ]:
import torch

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt")

# Generate the answer
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=32,         # Limit answer length
        do_sample=False            # Greedy output, or True for sampling
    )


In [ ]:
# Remove the prompt from the result to get only the answer
decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
# Remove the prompt if desired:
answer = decoded[len(prompt):].strip()
print("Answer:", answer)
